# Quantitative Example of Contributing Failure

<img src="../supportfiles/pipesetup1.png">

## System Failure Estimate

What is the failure rate, reliability, failure probability (unreliability), and MTBF of the pump system over one year?

Applicable Approximate Failure Rates:

| Component  | Failure Rate (λ\') (per $10^6$ hr) | Failure Rate (λ\') (per year) 
|------------|----------------|------------------------------------|
| Pump/Motor      | 10           | 0.0876                             |
| Control Valve   | 69        | 0.604                              |
| Flow measurement | 126         | 1.103                              |
| Controller      | 33          | 0.289                              |

### Recommended Method:

$$
P_{or} = 1 - \prod_{i=1}^{n} (1 - P_i)
$$

$$
P_{and} = \prod_{i=1}^{n} P_i
$$

$$
P_i = 1 - e^{-\lambda_i' t}
$$

The probability of the or condition is 1 minus the product of the probabilities of no failure for each component. The probability of the and condition is the product of the probabilities of failure for each component. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = {"Component": ["Pump/Motor", "Control Valve", "Flow measurement", "Controller"], "Failure Rate (λ') (per $10^6$ hr)":
         [10, 69, 126, 33], "Failure Rate (λ') (per year)": [0.0876, 0.604, 1.103, 0.289]}
df = pd.DataFrame(data)
df['Probability of Failure over 1 year'] = 1 - np.exp(-df['Failure Rate (λ\') (per year)'] * 1)
df

,Component,Failure Rate (λ') (per $10^6$ hr),Failure Rate (λ') (per year),Probability of Failure over 1 year
0,Pump/Motor,10,0.0876,0.083873
1,Control Valve,69,0.6040,0.453379
2,Flow measurement,126,1.1030,0.668126
3,Controller,33,0.2890,0.250988


Since if any one of the components fail, the system fails, we will use the OR method to calculate the overall system failure probability. Even though we don't know the failure rates that precisely, we won't truncate until after the final calculation which is good practice.

In [3]:
P_or = 1 - np.prod(1 - df['Probability of Failure over 1 year'])
print(f'P_or is {P_or:.2f}')

P_or is 0.88


The overall system failure rate can be found from the failure probability:
$$
\lambda_{system}' = -\frac{ln(1 - P_{or})}{t}
$$

In [4]:
sfr = -np.log(1 - P_or)/1 # per year
print(f'System failure rate λ\' is {sfr:.2f} per year')

System failure rate λ' is 2.08 per year


The Mean Time Between Failures (MTBF) is the inverse of the failure rate:
$$
MTBF = \frac{1}{\lambda_{system}'}
$$

In [5]:
print(f'MTBF is {1/sfr:.2f} years')

MTBF is 0.48 years


The reliability is simply 1 minus the failure probability. The unreliability is the failure probability itself.

### Other Methods:

Other methods can be used to determine the above result. For example you could sum the failure rates directly for an OR condition, or multiply them for an AND condition as shown below. Monte Carlo simulation is another alternative method that can be used to estimate system reliability and failure rates. You could also use fault tree analysis software to model the system and calculate the desired values. We'll talk more about these methods later in the course, but for an intro, below is a simple fault tree for this system.

In [6]:
#sum of the failure rates
sfr_sum = df['Failure Rate (λ\') (per year)'].sum()
sfp_sum = 1 - np.exp(-sfr_sum * 1)
print(f'Sum method: P_or is {sfp_sum:.2f}')

Sum method: P_or is 0.88


In [7]:
#product of the failure rates for AND condition
sfp_prod = np.prod(df['Probability of Failure over 1 year'])
sfp_and = 1 - np.exp(-sfp_prod * 1)
print(f'Product method: P_and is {sfp_and:.4f}')

Product method: P_and is 0.0064
